# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    spread_cls = []
    spread_avg = []
    
    tick_opn = []
    tick_hig = []
    tick_low = []
    tick_cls = []
    tick_avg = []
    tick_sd = []
    tick_ema_10 = []
    tick_ema_25 = []
    tick_ema_50 = []
    tick_ema_75 = []
    tick_ema = []
    tick_slope = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])      
        
        spread_cls.append(spread_list[-1])
        spread_avg.append(np.mean(spread_list))

        tick_opn.append(tick_list[0])
        tick_hig.append(np.max(tick_list))
        tick_low.append(np.min(tick_list))
        tick_cls.append(tick_list[-1])
        tick_avg.append(np.mean(tick_list))  
        tick_sd.append(np.std(tick_list))
        
        l = tick_list[-(int(len(tick_list)*0.10)):]
        tick_ema_10.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.25)):]
        tick_ema_25.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.50)):]
        tick_ema_50.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.75)):]
        tick_ema_75.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        tick_ema.append(list(pd.DataFrame(tick_list).ewm(span=len(tick_list)).mean()[0])[len(tick_list) - 1]) 
        tick_slope.append(get_slope(tick_list))
        
    temp_df['spread_avg'] = spread_avg  
    temp_df['spread_cls'] = spread_cls  

    temp_df['tick_opn'] = tick_opn
    temp_df['tick_high'] = tick_hig
    temp_df['tick_low'] = tick_low
    temp_df['tick_cls'] = tick_cls
    temp_df['tick_avg'] = tick_avg  
    temp_df['tick_sd'] = tick_sd
    
    temp_df['tick_ema_10'] = tick_ema_10  
    temp_df['tick_ema_25'] = tick_ema_25  
    temp_df['tick_ema_50'] = tick_ema_50  
    temp_df['tick_ema_75'] = tick_ema_75  
    temp_df['tick_ema'] = tick_ema 
    temp_df['tick_slope'] = tick_slope 

    return(temp_df)

In [3]:
def get_slope(y_axis):
    global data
    x_axis = []
    for i in range(len(y_axis)):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))

    return(slope_tick)

## File paths

In [4]:
year = 2019
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [5]:
data = {}
diff_col = 'sema'
data['pip_diff'] = 0.00002

data['number_of_ticks'] = 100

data['sma_len'] = 20
data['lma_len'] = 50

data['rsi_window'] = 14

In [ ]:
%%time
#df = pd.read_csv(source_file_path, nrows=1000000)
df = pd.read_csv(source_file_path)
df.head()

In [ ]:
%%time
df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')
df = pd.read_csv(chunk_file_path)
df.head()

In [ ]:
%%time

df['diff_next_high'] = df['tick_high'].shift(-1) - df['tick_cls']
df['diff_next_low'] = df['tick_low'].shift(-1) - df['tick_cls']

df['dir'] = 'same'
df.loc[abs(df['diff_next_high']) > abs(df['diff_next_low']), 'dir'] = 'high'
df.loc[abs(df['diff_next_low']) > abs(df['diff_next_high']), 'dir'] = 'low'

df['direction'] = 'same'
df.loc[(df['dir'] == 'high') & (df['diff_next_high'] >= 0.0002), 'direction'] = 'increase'
df.loc[(df['dir'] == 'high') & (df['diff_next_high'] <= -0.0002), 'direction'] = 'decrease'

df.loc[(df['dir'] == 'low') & (df['diff_next_low'] >= 0.0002), 'direction'] = 'increase'
df.loc[(df['dir'] == 'low') & (df['diff_next_low'] <= -0.0002), 'direction'] = 'decrease'

In [ ]:
%%time
col_list = ['tick_opn', 'tick_high', 'tick_low', 'tick_cls', 'tick_avg', 'tick_ema', 'spread_cls', 'tick_ema_10', 'tick_ema_25', 'tick_ema_50', 'tick_ema_75']
for i, val in enumerate(col_list):    
    df[val+'_diff'] =  df[val].diff()
    del df[val]    
    
df['ema_gap'] = df['tick_ema_10_diff'] - df['tick_ema_diff']    

In [ ]:
%%time
data['rs_max'] = 1e6
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['tick_avg_diff'] > 0] = abs(df['tick_avg_diff'])
df['loss'].loc[df['tick_avg_diff'] < 0] = abs(df['tick_avg_diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

In [ ]:
col_order = [
 'spread_avg',
 'spread_cls_diff', 
 'tick_opn_diff',
 'tick_high_diff',
 'tick_low_diff',
 'tick_cls_diff',
 'tick_avg_diff',
 'tick_sd',
 'tick_ema_10_diff',
 'tick_ema_25_diff',
 'tick_ema_50_diff',
 'tick_ema_75_diff',
 'tick_ema_diff',
 'ema_gap',
 'gain',
 'loss',
 'avg_gain',
 'avg_loss',
 'rsi',
 'tick_slope',
 'direction'
 ]

df = df[col_order]
df = df.dropna()

## Write data to csv

In [ ]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

In [ ]:
print(f'Total records : {len(df)}')
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head()